In [2]:
import pandas as pd
import numpy as np
import os, sys

from sklearn.decomposition import PCA
from numpy.linalg import pinv, eig

from pandas.tseries.offsets import BDay, Day, MonthEnd, DateOffset
from sklearn.linear_model import LassoLars, Lars, LassoLarsCV
from sklearn.model_selection import TimeSeriesSplit
import statsmodels.api as sm

parentdir = "/Users/paoloandreini/Desktop/github_repo/DDFM_correction_paper/DDFM/"
sys.path.insert(0, parentdir)

from models.bai_ng_models import TargetedPredictiors, BaiNgModels
from tools.bai_ng_utils import transform_variables, preprocess_data, rename_col_factors

import warnings
warnings.filterwarnings("ignore")

chekc and modify parentdir


In [3]:
data_m_all = pd.read_csv(f"{parentdir}data/mdfred_snapshot_monthly.csv")
transform_code_m = data_m_all.iloc[0,1:]
data_m = data_m_all.iloc[1:,:].set_index("sasdate")
data_m.index = pd.to_datetime(data_m.index)
# convert to month end
data_m.index = data_m.index + MonthEnd()

data_q_all = pd.read_csv(f"{parentdir}data/mdfred_snapshot_quarterly.csv")
transform_code_q = data_q_all.iloc[1,:]
data_q = data_q_all.iloc[2:,:].set_index("sasdate")
data_q.index = pd.to_datetime(data_q.index)
# convert to month end and shift 1 month - pseudo real-time
# data_q.index = data_q.index + DateOffset(months=1) 
data_q.index = data_q.index + MonthEnd()

# # keep only gdp
gdp_q = data_q["GDPC1"]

# merge it with monthly data
data_final = pd.concat([gdp_q, data_m], axis=1)
transform_code_final = transform_code_m
transform_code_final.loc["GDPC1"] = transform_code_q["GDPC1"]
transform_code_final = transform_code_final.astype(int)

In [4]:
# pca_type:
# classic PCA: PC and PC2
# LarsPCA: LA - 5 variables, LAPC - 30 variables from X, LASPC 30 variables form X and X**2
# targeted predictors: TPC- targeted preditors from X, and TPC2 - target targeted preditors from X adn X**2
model_names = ["PC", "SPC", "SPC2", "TPC", "TSPC", "TPC2", "PC2", "LA5", "LAPC"]
forecast={"backcast":1, "nowcast": 4, "forecast":7, "forecast2s": 11}

In [5]:
# re-estimate once a yaer
months_oos = 12*19
test_size = 12
n_splits = int(months_oos/test_size)

# if wnat oly to standardize data
# pre-process data: staz and stand
# model_name = "" # can leave it blacnk
#staz = True
#standardize=True
#start_date = pd.Timestamp("1990-01-01")
# data, mean, sigma = preprocess_data(data, transform_code_final, target_name, model_name, staz, standardize, start_date)
# X_all = data.drop(target_name, axis=1)
# y_all= data[target_name]

In [33]:
# loop on number of factors??
y_pred_all_models = {}
for fcst_name, fcst_lag in forecast.items():
    y_pred_all_tmp = []
    for name_i in model_names:
        print(name_i, fcst_lag)
        bai_ng_mdl = BaiNgModels(data_final, transform_code_final,
                    staz=True, standardize=True, target_name=None,
                    start_date=pd.Timestamp("1990-01-01"),
                    # models
                    n_factors=3, # maybe try 5 and 10 as they do in the paper / or we shold use the same value we use
                    model_name = name_i,
                    lars_coef_number=None,
                    target_freq=None,
                    thresh_tstat= 1.65, # we should use 1.65, 1.96, 2.56
                    n_targeted_predictors=None,    
                    lags_y = 5, # 5 is actually 4
                    lags_f = 7, # 7 is actually 6):
                    )
        X_all = bai_ng_mdl.X_all.shift(1).bfill()
        y_all = bai_ng_mdl.y_all.shift(fcst_lag).bfill() # shift ahead by to make it "pseudo-real" time
        target_mean = bai_ng_mdl.mean.shift(fcst_lag)
        target_std = bai_ng_mdl.sigma.shift(fcst_lag)
        target_name = bai_ng_mdl.target_name
        tscv = TimeSeriesSplit(max_train_size=None, n_splits=n_splits, test_size=test_size)
        tscv_index = tscv.split(X_all)

        y_pred_all = []
        for i, (train_index, test_index) in enumerate(tscv_index):
            X = X_all.iloc[train_index]
            y = y_all.iloc[train_index]

            # fit model
            bai_ng_mdl.fit(X, y)

            # predict
            # need to create X_reg
            test_index_factors = list(train_index) + list(test_index)
            X_pred = X_all.iloc[test_index_factors]
            y_pred = y_all.iloc[test_index_factors]
            y_pred = bai_ng_mdl.predict(X_pred, y_pred)
            test_index_date = y_all.iloc[test_index].index
            # print(test_index_date)
            y_pred_all.append(y_pred.loc[test_index_date].map(np.real))
        y_pred_all = pd.DataFrame(pd.concat(y_pred_all), columns = [name_i])
        y_pred_all_tmp.append(y_pred_all)
        
        y_pred_all_models_df = pd.concat(y_pred_all_tmp, axis=1)
        # convert to actual staz values not standardize
        y_pred_final = ((y_pred_all_models_df.T *  target_std[target_name]) + target_std[target_name]).T
        
    y_pred_all_models[fcst_name] = y_pred_final.dropna(how="all", axis=1)
        

PC 1
SPC 1
SPC2 1
TPC 1
TSPC 1
TPC2 1
PC2 1
LA5 1
LAPC 1
PC 4
SPC 4
SPC2 4
TPC 4
TSPC 4
TPC2 4
PC2 4
LA5 4
LAPC 4
PC 7
SPC 7
SPC2 7
TPC 7
TSPC 7
TPC2 7
PC2 7
LA5 7
LAPC 7
PC 11
SPC 11
SPC2 11
TPC 11
TSPC 11
TPC2 11
PC2 11
LA5 11
LAPC 11


In [34]:
# compute RMSE
start_date = "2006-01-01"
end_date = "2020-05-01"
y_true = (bai_ng_mdl.y_all * bai_ng_mdl.sigma[target_name]) + bai_ng_mdl.mean[target_name]

rmsfe_all = {}
for fcst_name, fcst_lag in forecast.items():
    y_hat = y_pred_all_models[fcst_name].dropna(how="all", axis=1) 
    rmsfe = np.sqrt((((y_hat.T - y_true.shift(fcst_lag)).T)**2).loc[start_date:end_date].mean())
    rmsfe_all[fcst_name]=rmsfe


In [35]:
rmsfe_df = pd.concat(rmsfe_all, axis=1)

In [38]:
# extrapolated values from our chart - did not find proper excel with all results.
rmsfe_df.loc["DFM3F", :] = [0.53, 0.53, 0.64, np.nan]
rmsfe_df.loc["DDFM", :] = [0.52, 0.5050, 0.60, np.nan]
rmsfe_df.loc["AR", :] = [0.64, 0.65, 0.6950, np.nan]

In [39]:
rsmfe_wrt_ar = rmsfe_df / rmsfe_df.loc["AR"]
rsmfe_wrt_ar.sort_values(by="backcast")

,backcast,nowcast,forecast,forecast2s
TPC2,0.785255,0.715724,0.767118,NaN
DDFM,0.812500,0.776923,0.863309,NaN
TPC,0.812793,0.798232,0.802458,NaN
DFM3F,0.828125,0.815385,0.920863,NaN
LA5,0.862558,0.781798,0.782989,NaN
AR,1.000000,1.000000,1.000000,NaN
LAPC,1.150133,1.146432,1.231356,NaN
PC2,1.299397,1.927577,1.849507,NaN
PC,1.409700,1.390676,1.238133,NaN
SPC,1.652435,1.774611,1.432459,NaN


In [40]:
rmsfe_df

,backcast,nowcast,forecast,forecast2s
PC,0.902208,0.903939,0.860502,0.715683
SPC,1.057558,1.153497,0.995559,0.838475
SPC2,1.057558,1.153497,0.995559,0.838475
TPC,0.520187,0.518851,0.557708,0.849024
TSPC,1.057558,1.153497,0.995559,0.838475
TPC2,0.502564,0.465221,0.533147,1.814334
PC2,0.831614,1.252925,1.285407,0.871146
LA5,0.552037,0.508169,0.544177,0.602207
LAPC,0.736085,0.745181,0.855792,0.731695
DFM3F,0.530000,0.530000,0.640000,NaN


In [41]:
rmsfe_df

,backcast,nowcast,forecast,forecast2s
PC,0.902208,0.903939,0.860502,0.715683
SPC,1.057558,1.153497,0.995559,0.838475
SPC2,1.057558,1.153497,0.995559,0.838475
TPC,0.520187,0.518851,0.557708,0.849024
TSPC,1.057558,1.153497,0.995559,0.838475
TPC2,0.502564,0.465221,0.533147,1.814334
PC2,0.831614,1.252925,1.285407,0.871146
LA5,0.552037,0.508169,0.544177,0.602207
LAPC,0.736085,0.745181,0.855792,0.731695
DFM3F,0.530000,0.530000,0.640000,NaN
